
#Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

+++++++++++++++++++++++++++++++++++++++++++++++++

Name : Name of the latest contact at Company

Age: Customer Age

Total_Purchase: Total Ads Purchased

Account_Manager: Binary 0=No manager, 1= Account manager assigned

Years: Totaly Years as a customer

Num_sites: Number of websites that use the service.

Onboard_date: Date that the name of the latest contact was onboarded

Location: Client HQ Address

Company: Name of Client Company

++++++++++++++++++++++++++++++++++++++++++++++++++


Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark.sql import SparkSession

#Load the data

In [ ]:
spark = SparkSession.builder.appName('mylogreg').getOrCreate()

In [ ]:
data = spark.read.csv('customer_churn.csv', header=True, inferSchema=True)

In [ ]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [ ]:
 for item in data.head(1)[0]:
   print(item)

Cameron Williams
42.0
11066.8
0
7.22
8.0
2013-08-30 07:00:40
10265 Elizabeth Mission Barkerburgh, AK 89518
Harvey LLC
1


In [ ]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

#Preparing for MLlib

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'], outputCol='features')

In [ ]:
my_data = assembler.transform(data).na.drop()
my_data.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+--------------------+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|            features|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+--------------------+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|[42.0,11066.8,0.0...|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|[41.0,11916.22,0....|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|[38.0,12884.75,0....|
|      Phillip White|4

In [ ]:
my_final_data = my_data.select('features','churn')

#Splitting the data

In [ ]:
train_data, test_data = my_final_data.randomSplit([0.7,0.3])

#Fit the model

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
lr = LogisticRegression(labelCol='churn')

In [ ]:
fitted_lr = lr.fit(train_data)

In [ ]:
training_sum = fitted_lr.summary

In [ ]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                635|                635|
|   mean|0.17795275590551182|0.13543307086614173|
| stddev| 0.3827744857128543|0.34245531128822343|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



#Evaluate results

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
pred_and_labels = fitted_lr.evaluate(test_data)

In [ ]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8787.39,1.0...|    1|[0.43430708300304...|[0.60690168970891...|       0.0|
|[26.0,8939.61,0.0...|    0|[6.01725526600237...|[0.99756957381071...|       0.0|
|[27.0,8628.8,1.0,...|    0|[5.03623938224676...|[0.99354381375326...|       0.0|
|[28.0,8670.98,0.0...|    0|[7.41539606619338...|[0.99939844813433...|       0.0|
|[28.0,11245.38,0....|    0|[3.28387551681242...|[0.96387148458705...|       0.0|
|[29.0,8688.17,1.0...|    1|[2.39056093201608...|[0.91610468965487...|       0.0|
|[30.0,6744.87,0.0...|    0|[3.24698776200838...|[0.96256472114712...|       0.0|
|[30.0,8874.83,0.0...|    0|[2.8810984397299,...|[0.94690411665627...|       0.0|
|[30.0,10183.98,1....|    0|[2.62129553274506...|[0.93221961207512...|       0.0|
|[30.0,10960.52,

Using AUC:

In [ ]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [ ]:
auc = churn_eval.evaluate(pred_and_labels.predictions)
auc

0.7866880037932668

#Predict on brand new unlabeled data

In [ ]:
final_lr_model = lr.fit(my_final_data)

In [ ]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,
                              header=True)

In [ ]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [ ]:
test_new_customers = assembler.transform(new_customers)

In [ ]:
final_results = final_lr_model.transform(test_new_customers)

In [ ]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

